In [2]:
import sys
import os
dir_path = os.path.dirname(os.path.abspath('.'))
print(dir_path)
sys.path.append(dir_path)

/media/disk1/chatgpt/zh/tabular_data


In [4]:
import pandas as pd
from utils import parse_specific_composition, add_row_number
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from data_loader import TableFormat, TableLoader
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from sqlalchemy import create_engine
from executor import SQLManager
embeddings = HuggingFaceBgeEmbeddings(
            model_name='BAAI/bge-large-en',
            model_kwargs={'device': 'cuda:0', 'trust_remote_code': True},
            encode_kwargs={'normalize_embeddings': True})

### Prompts Predefined

In [6]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = [
                  "what was the time difference between the first place finisher and the eighth place finisher?",
                  "other than william stuart price, which other businessman was born in tulsa?",
                  "which canadian city had the most passengers traveling from manzanillo international airport in 2013?"
                  ]
new_query_examples = [
                      "what was the time for the first place finisher?; what was the time for the eighth place finisher?",
                      "was william stuart price born in tulsa?; who was born in tulsa?",
                      "how many passengers do each airline from canadian city have?; which canadian city had the most passengers?"
                      ]
num_k = 3
inds = [1, 11, 86]
table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
normalised_data = [table_loader.normalize_table(table_loader.dataset[inds[i]]) for i in range(num_k)]
example_samples = [TableFormat(format='none', data=normalised_data[i], save_embedding=True, embeddings=embeddings).get_sample_data(sample_type='embedding', query=normalised_data[i]['query']) for i in range(num_k)]
examples = [TableFormat.format_html(example_samples[i], normalised_data[i]['table']['caption']) for i in range(num_k)]

examples_prompt = PromptTemplate(input_variables=["query", "table", "new_query"], template=
"""Sub-Table: {table}
Query: {query}
Decompose query: {new_query}""")

examples_dict = [{"query": query_examples[i],
                                    "table": examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
decompose_prompt_wiki = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are capable of converting complex query into sub-queries. Below is a sub-table with rows randomly sampled from the original table. Based on the sub-table, decompose the original query into 2-3 complete sub-queries that can solve the original query.""",
    suffix=
    """Sub-Table: {table}
Query: {query}
Decompose query: """,
    input_variables=["query", "table"],
)



In [9]:
task_name = 'wikitable'
split = 'test'
model_name = 'gpt-3.5-turbo'
schema_information = pd.read_csv(f"../result/augmentation/{task_name}_{split}_schema.csv", index_col='table_id')
aug_information = pd.read_csv(f"../result/augmentation/{task_name}_{split}_summary.csv", index_col='table_id')
composition_information = pd.read_csv(f"../result/augmentation/{task_name}_{split}_composition.csv", index_col='table_id')
engine = create_engine('sqlite:////media/disk1/chatgpt/zh/tabular_data/db/sqlite/cell.db', echo=False)
manager = SQLManager(engine=engine)

In [10]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=False, small_test=False)
i = 2


def show_table(data, execute=False):
    formatter = TableFormat(format='none', data=data)
    print(data['id'])
    print(data['query'])
    print(formatter.format_psql())
    # print(preds[i])
    # print(SQLs[i])
    # test_df = manager.execute_from_df(SQLs[i], formatter.all_data)
    # print(test_df)
    print(data['label'])
print(table_loader.dataset[i]['caption'])
show_table(table_loader.normalize_table(table_loader.dataset[i]) )

New York Americans (soccer)
nu-2
how long did it take for the new york americans to win the national cup after 1936?
+-------------+------------+----------+------------------+-----------------------+----------------+
| Year        | Division   | League   | Reg_Season       | Playoffs              | National_Cup   |
|-------------+------------+----------+------------------+-----------------------+----------------|
| 1931        | 1          | ASL      | 6th (Fall)       | No playoff            | N/A            |
| Spring 1932 | 1          | ASL      | 5th?             | No playoff            | 1st Round      |
| Fall 1932   | 1          | ASL      | 3rd              | No playoff            | N/A            |
| Spring 1933 | 1          | ASL      | ?                | ?                     | Final          |
| 1933/34     | N/A        | ASL      | 2nd              | No playoff            | ?              |
| 1934/35     | N/A        | ASL      | 2nd              | No playoff            | 

In [11]:
table_loader = TableLoader(table_name=task_name, split='test', use_sample=False, small_test=False)
sample = table_loader.normalize_table(table_loader.dataset[2])

In [13]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
inds = [11, 182, 70]
num_k = 2
table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
normalised_data = [table_loader.normalize_table(table_loader.dataset[inds[i]]) for i in range(num_k)]
example_samples = [TableFormat(format='none', data=normalised_data[i], save_embedding=True,embeddings=embeddings).get_sample_data(sample_type='embedding', query=normalised_data[i]['query']) for i in range(num_k)]
examples = [TableFormat.format_html(example_samples[i], normalised_data[i]['table']['caption']) for i in range(num_k)]
new_query_examples = [
    # "Which country uses the US dollar as its currency and has the Federal Reserve as its central bank?",
    "which business man was born in tulsa?",
    "what is the network owned by national polytechnic institute?",
    "what districts are more populous than haridwar?"
    ]
examples_prompt = PromptTemplate(input_variables=["query", "new_query"], template=
"""Sub-table: {table}
Query: {query}
New query: {new_query}""")

examples_dict = [{"query": table_loader.dataset[inds[i]]['question'],
                  "table": examples[i],
                    "new_query": new_query_examples[i]} for i in range(num_k)]
step_back_prompt_wiki = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""Below is a sub-table with rows randomly sampled from the original table. Based on the sub-table, your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.""",
    suffix=
    """Sub-table: {table}
Query: {query}
New query: """,
    input_variables=["query", "table"],
)

In [14]:
def get_k_shot_with_answer(k: int=1):
        sqls = ["SELECT DISTINCT Type FROM DF WHERE Type != 'audio';"
                ]
        thoughts = ["Based on the SQL query and the extra information provided, the types include audio or video. Therefore, other than audio, the payload type is video."]
        tables = ['<table>\n<thead>\n<tr><th> Type </th></tr>\n</thead>\n<tbody>\n<tr><td>video   </td></tr>\n<tr><td>audio/video   </td></tr>\n</tbody>\n</table>']
        claims = ["other than audio, what type of payload types are there?"]
        extras = ["The payload types for audio include audio, video, and audio/video."]
        examples_prompt = PromptTemplate(input_variables=["SQL", "table", "information",  "claim", "thought", "output"], template=
        """
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Extra information:
{information}

Query: {claim}
Thought: {thought}
Answer: {output}
        """)
        examples_dict = dict(zip(["SQL", "table", "information",  "claim", "thought", "output"], [sqls[0], tables[0], extras[0], claims[0], thoughts[0], 'video']))
        prompt_template = FewShotPromptTemplate(
                examples=[examples_dict],
                example_prompt=examples_prompt,
                prefix="""Below is a sub-table generated by executing the corresponding SQL. You need to understand the logic behind the SQL filtering. Think step by step and answer the question given in the query.
You should output in the following format:
Thought: your step by step thought
Answer: Only return the concise string instead of other format information. Do not repeat the question.
Below is an example.""",
        suffix=
        """
SQL Executed: 
```{SQL}```
Sub-table: {table}
Extra information:
{information}

Query: {query}""",
                input_variables=["table", "query", "SQL", "information"],
        )
        return prompt_template

In [15]:
def get_k_shot_with_aug(k: int=2):
    table_loader = TableLoader(table_name='wikitable', split='train', use_sample=True, small_test=False)
    examples_dict = []
    
    examples_dict.extend([{"table": '<table>\n<caption>Hoot Kloot</caption>\n<thead>\n<tr><th> Number</th><th> Title</th><th> Directed_by_</th><th> Released_</th></tr>\n</thead>\n<tbody>\n<tr><td>1  </td><td>"Kloot\'s Kounty"           </td><td>Hawley Pratt  </td><td>1973       </td></tr>\n<tr><td>2  </td><td>"Apache on the County Seat"</td><td>Hawley Pratt  </td><td>1973       </td></tr>\n<tr><td>6  </td><td>"Mesa Trouble"       </td><td>Sid Marcus </td><td>1974       </td></tr>\n</tbody>\n</table>',
                                        "claim": table_loader.dataset[95]['question'],
                                        "aug": "The table contains information about the Hoot Kloot animated series, including the episode number, title, director, and release year. \n1. Number: The episode number in the series \n2. Title: The title of the episode \n3. Directed_by_: The director of the episode \n4. Released_: The release year of the episode",
                                        "linking": "the last title -> Released_, the last title-> Number, title -> Title, sid marcus -> Directed_by_",
                                        "output": "Released_, Number, Title, Directed_by_"}])
    examples_dict.extend([{"table": '<table>\n<caption>1943–44 Chicago Black Hawks season</caption>\n<thead>\n<tr><th>  num</th><th>       Date</th><th>            Visitor</th><th>  Score</th><th>               Home</th><th>  Record</th></tr>\n</thead>\n<tbody>\n<tr><td>14   </td><td>December 5 </td><td>New York Rangers   </td><td>6–7    </td><td>Chicago Black Hawks</td><td>8–6–0   </td></tr>\n<tr><td>40   </td><td>February 26</td><td>Chicago Black Hawks</td><td>3–2    </td><td>Toronto Maple Leafs</td><td>18–18–4 </td></tr>\n<tr><td>31   </td><td>January 29 </td><td>Chicago Black Hawks</td><td>4–3    </td><td>Toronto Maple Leafs</td><td>14–16–1 </td></tr>\n</tbody>\n</table>',
                                        "claim": 'what was the difference in score in the december 19th win?',
                                        "aug": 'The table contains information about the 1943-44 Chicago Black Hawks season, including the date, visitor, score, home team, record, and points for each game. \n1. num: The game number in the season \n2. Date: The date of the game\n3. Vistor: The visiting team\n4. Score: The final score, with the visitor score listed first\n5. Home: The home team\n6. Record: The team win-loss-overtime loss record at the time of the game',
                                        "linking": 'difference in score -> Score, december 19th -> Date',
                                        "output": 'Date, Score'}])
    
    # "The table contains information about the Hoot Kloot animated series, including the episode number, title, director, and release year."
    # "№<The episode number in the series>\nTitle<The title of the episode>\nDirected_by_<The director of the episode>\nReleased_<The release year of the episode>"
    examples_prompt = PromptTemplate(input_variables=["table", "aug","claim", "output", "linking"], template=
    """
Table: {table}
Extra information: {aug}

Query: {claim}
Column linking: {linking}
Columns: {output}""")
    prompt_template = FewShotPromptTemplate(
        examples=examples_dict,
        example_prompt=examples_prompt,
        prefix=
        """
Based on the Table below, your task is to accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the Table.
Approach this task as follows:
Read the query and extra information thoroughly and list every possible link from query term to column in the Table. 
Then based on the column linking, output all useful columns at last. Make sure all columns in the linking step are included and every column is in the Table.""",
        suffix=
        """
Table: {table}
Extra information: {aug}

Query: {claim}""",
        input_variables=["table", "claim", "aug"],
)
    return prompt_template




In [16]:
from utils import parse_output
def scene_A(query, sample, k =3, verbose=True):

    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                    template="""Our ultimate goal is to answer the query based on the original table. Now we have a sub-table with rows randomly sampled from the original table, you are required to infer the data distribution and format from the sample data of the sub-table.  Based on the augmentation information, carefully analyze the query and write an SQLITE3 SELECT SQL statement using table DF that completes the query. Directly Output SQL, do not add other string.
sub-table: {table}
Extra information: {aug}

Query: {claim}
SQL: """)
    formatter = TableFormat(format='none', data=sample, save_embedding=True, embeddings=embeddings)
    k_shot_prompt = get_k_shot_with_aug()
    formatter.normalize_schema(schema_information.loc[sample['table']['id']]['schema'])
    if k == 0:
        sample_data = formatter.get_sample_data(sample_type='head', k=k)
    else:
        sample_data = formatter.get_sample_data(sample_type='embedding', query=query, k=k)
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['table']['id']]['summary'], aug_information.loc[sample['table']['id']]['column_description'] 
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        
        stage_1_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug +'\n'+ '\n'.join(extra_col_info)
                                            # 'aug': ''
                                            })], return_only_outputs=True)[0]['text']
        if verbose:
            print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        extra_cols = formatter.get_sample_column(embeddings, column_aug)
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = list(set([c.strip() for c in stage_1_batch_pred.split(',')] + extra_cols))
        try: 
            # formatter.all_data = formatter.all_data.loc[:, columns]
            sample_data = add_row_number(sample_data.loc[:, columns])
        except:
            sample_data = add_row_number(sample_data)
        extra_information = []
        tuples = parse_specific_composition(composition_information.loc[sample['table']['id']]['composition'], sample_data.columns)
        for col, com in tuples:
            if len(pd.unique(formatter.all_data[col])) < 6:
                com += f' (Values like {", ".join(list(formatter.all_data[col].dropna().unique().astype(str)))})'
                extra_information.append(col + ':' + com)
            else:
                com += f' (Values like {", ".join(list(formatter.all_data[col].dropna().unique()[:3].astype(str)))}...)'
                extra_information.append(col + ':' + com)
        extra_information.append('row_number: row index in the original table')
        stage_2_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data = sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\nColumn information:\n' + '\n'.join(extra_information)
                                            # 'aug': ''
                                            })], return_only_outputs=True)[0]['text'].replace("–", "-").replace("—", "-").replace("―", "-").replace("−", "-")
        if verbose:
            print(stage_2_batch_pred)
    # stage 3: SQL Excution
    try: 
        execute_data = manager.execute_from_df(stage_2_batch_pred, add_row_number(formatter.all_data), table_name='DF')
    except:
        execute_data = formatter.all_data
        stage_2_batch_pred = 'SELECT * from DF;'
    if len(execute_data) == 0:
        return query, stage_2_batch_pred, 'No data from database', cb.total_tokens
    return query, stage_2_batch_pred, TableFormat.format_html(data=execute_data), cb.total_tokens
    # return query, stage_2_batch_pred, execute_data, cb.total_tokens

In [17]:
import concurrent.futures
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)

In [18]:
def eval_blury_string(pred_list):
    pred_label = []
    for pred in pred_list:
        predict_ans = pred.split('\n')[-1]
        if '0' in predict_ans:
            predict_ans = '0'
        elif '1' in predict_ans:
            predict_ans = '1'
        else:
            predict_ans = '2'
        pred_label.append(predict_ans)
    return pred_label

In [19]:
import concurrent.futures
from typing import List
import os
import json
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)

def save_csv(input_list: List[List], label_list: List, file_path):
    import pandas as pd
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    assert len(input_list) == len(label_list)
    df = pd.DataFrame()
    for i in range(len(label_list)):
        df[label_list[i]] = pd.Series(input_list[i])
    if os.path.exists(file_path) and file_path.endswith('.csv'):
        df_origin = pd.read_csv(file_path)
        df = pd.concat([df_origin, df], axis=0)
    df.to_csv(file_path, index=False, encoding='utf-8')

In [20]:
from utils import parse_output
answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
                                    template="""
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering and answer the query using the final sub-table. 
SQL Excuted: 
```{SQL}```
Sub-table: 
{table}
Query: {claim}
Please provide a clear, complete statement in response to the query. If you cannot answer the query based on the sub-table, just say 'Cannot get answer from sub-table'.
""" )
def scene_B(query, sample, k=3, verbose=False):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""Our ultimate goal is to answer the query based on the original table. Now we have a sub-table with rows randomly sampled from the original table, you are required to infer the data distribution and format from the sample data of the sub-table.  Based on the augmentation information, carefully analyze the query and write an SQLITE3 SELECT SQL statement using table DF that completes the query. Directly Output SQL, do not add other string.
sub-table: {table}
Extra information: {aug}

Query: {claim}
SQL: """)
    
    formatter = TableFormat(format='none', data=sample, save_embedding=True, embeddings=embeddings)
    formatter.normalize_schema(schema_information.loc[sample['table']['id']]['schema'])
    if k == 0:
        sample_data = formatter.get_sample_data(sample_type='head', k=k)
    else:
        sample_data = formatter.get_sample_data(sample_type='embedding', query=query, k=k)
    # get columns
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_aug(), verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['table']['id']]['summary'], aug_information.loc[sample['table']['id']]['column_description'] 
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        stage_1_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n' + '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        extra_cols = formatter.get_sample_column(embeddings, column_aug)
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = list(set([c.strip() for c in stage_1_batch_pred.split(',')] + extra_cols))
        try: 
            sample_data = add_row_number(sample_data.loc[:, columns])
        except:
            sample_data = add_row_number(sample_data)
        extra_information = []
        tuples = parse_specific_composition(composition_information.loc[sample['table']['id']]['composition'], sample_data.columns)
        for col, com in tuples:
            if len(pd.unique(formatter.all_data[col])) < 6:
                com += f' (Values like {", ".join(list(formatter.all_data[col].dropna().unique().astype(str)))})'
                extra_information.append(col + ':' + com)
            else:
                com += f' (Values like {", ".join(list(formatter.all_data[col].dropna().unique()[:3].astype(str)))}...)'
                extra_information.append(col + ':' + com)
        #  sample augmentation
        # extra_information = (parse_specific_composition(composition_information.loc[sample['table']['id']]['composition'], sample_data.columns))
        extra_information.append('row_number: row index in the table')
        stage_2_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n Column information:' + '\n'.join(extra_information)
                                            })], return_only_outputs=True)[0]['text'].replace("–", "-").replace("—", "-").replace("―", "-").replace("−", "-")
    
        
        # stage 3: SQL Excution
        try: 
            execute_data= manager.execute_from_df(stage_2_batch_pred, add_row_number(formatter.all_data), table_name='DF')
        except:
            execute_data = formatter.all_data
            stage_2_batch_pred = 'SELECT * from DF;'
        llm_chain = LLMChain(llm=model, prompt=answer_instruction, verbose=verbose)
        response = llm_chain.batch([dict({'table': TableFormat.format_html(execute_data),
                                                'claim': query,
                                                'SQL':  stage_2_batch_pred
                                                })], return_only_outputs=True)[0]['text']
    # print("total_tokens:", cb.total_tokens)
    return response, cb.total_tokens

### Pipeline

In [23]:
from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_base="API_BASE",
                       openai_api_key="sk-api-key", temperature=0.01)

In [ ]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
from openai import BadRequestError
from tqdm.notebook import tqdm
# table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=False, small_test=False)
save_path = f"../result/final_answer/wikitable_{datetime.datetime.now().strftime('%m-%d_%H-%M-%S')}.csv"
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
# template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
template = """
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task with the help of extra information below.

SQL Excuted: 
```{SQL}```
Sub-table:
{table}
Extra information:
{information}

Query: {query}
Think step by step and answer the last question given in the query. Only return the string instead of other format information. Do not repeat the question.
""" )
sample_k = 3
# Task: answer the last question given in the query. Only return the string instead of other format information. Do not repeat the question.
# Task: verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.


# muilti_answer_instruction = get_k_shot_with_answer()
# for sample_n in range(3):
tokens = []
outputs = []
labels = []
ids = []
extra_quries = []
i = 0
with tqdm(total=len(table_loader.dataset), desc=f"Processing",ncols=1500) as pbar:
    while i < len(table_loader.dataset):
        try:
            sample = table_loader.normalize_table(
                                table_loader.dataset[i])
            all_tokens = 0
            all_queries = []
            formatter = TableFormat(format='none', data=sample, save_embedding=False)
            sample_data = formatter.get_sample_data(sample_type='random', k=sample_k, query=sample['query'])
            with get_openai_callback() as cb:
                llm_chain = LLMChain(llm=model, prompt=step_back_prompt_wiki, verbose=False)
                batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_html(sample_data)}], return_only_outputs=True)
                if batch_pred[0]['text'].strip() != sample['query']:
                    all_queries.append(batch_pred[0]['text'].strip())
                llm_chain = LLMChain(llm=model, prompt=decompose_prompt_wiki, verbose=False)
                batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_html(sample_data)}], return_only_outputs=True)
                all_queries.extend([q.strip() for q in batch_pred[0]['text'].split(';')])
                # print(all_queries)
            all_tokens += cb.total_tokens
            all_queries = list(set(all_queries))
            args_list = [{"query": q, "sample": sample, "k": sample_k} for q in all_queries]
            # print(args_list)
            ans_from_B = parallel_run_kwargs(scene_B, args_list)
            results = [res[0] for res in ans_from_B if 'Cannot get answer from sub-table' not in res[0] ]
            all_tokens += sum([res[1] for res in ans_from_B])
                
            with get_openai_callback() as cb:
                imp_input = scene_A(sample['query'], sample, sample_k, True)
                llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_answer(), verbose=True)
                batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(results)}], return_only_outputs=True)
            print(batch_pred[0])
            all_tokens += cb.total_tokens
            # print('ALL TOKENS', all_tokens)
            ids.append(sample['id'])
            labels.append(sample['query'])
            outputs.append(batch_pred[0]['text'])
            tokens.append(all_tokens)
            extra_quries.append(';'.join(all_queries))
            if (i + 1) % 10 == 0:
                    print(f'saving {i}')
                    save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)
                    outputs = []
                    labels = []
                    ids = []
                    tokens = []
                    extra_quries = []
            i += 1
            print(f' Process {i}')
            pbar.update(1)
        
        except BadRequestError as e:
            print('*************************Bad Request**************')
            i += 1
            pbar.update(1)
        except ValueError as e:
            print(f'******************Value Error {i}****************************')
            i += 1
            pbar.update(1)

    

